# Knee Joint Center Analysis - Anatomical Landmark Method

This notebook analyzes knee joint biomechanics using anatomical bone landmarks (not cluster markers).

## Data Structure
- Input: CSV files from motion capture system with anatomical markers
- Location: `data/*_Bone Model.csv` files

## Anatomical Markers Used

### Thigh/Femur Markers:
- MFEC: Medial Femoral Epicondyle
- LFEC: Lateral Femoral Epicondyle
- FH: Femoral Head
- MC: Greater Trochanter/Medial Condyle

### Shin/Tibia Markers:
- MTC: Medial Tibial Condyle
- LTC: Lateral Tibial Condyle
- uTD: Upper Tibia (Tuberosity)
- MM: Medial Malleolus (ankle)
- LM: Lateral Malleolus (ankle)

## Methods by Trial Type
- Static trials: Knee joint center (COR) is defined anatomically as the midpoint between MFEC and LFEC (averaged across frames to reduce noise). Sphere fitting is not used for static data.
- Dynamic trials: COR is estimated by sphere fitting the epicondyle trajectories (functional method).

  ## Coordinate Systems

  To analyze the knee's motion, we define local coordinate systems fixed to
  the thigh (femur) and shin (tibia) segments. These frames move with each
  bone and let us express the knee joint center in anatomically meaningful
  directions. The origin and axes are recomputed at every frame from the 3D
  marker positions.

  ### Thigh (Femur) Coordinate System
  - **Origin:** Lateral Femoral Epicondyle (LFEC) marker.
  - **Axes:**
    - **X-axis (Medial–Lateral):** From LFEC toward the Medial Femoral
  Epicondyle (MFEC).
    - **Y-axis (Proximal–Distal):** Points proximally along the femur’s long
  axis toward the hip.
    - **Z-axis (Anterior–Posterior):** Points posteriorly; perpendicular to the
  plane defined by the X- and Y-axes.
  - **Interpretation:**
    - `+X` medial (toward the contralateral leg)
    - `+Y` proximal (toward the hip)
    - `+Z` posterior (backward)

  ### Shin (Tibia) Coordinate System
  - **Origin:** Lateral Tibial Condyle (LTC) marker.
  - **Axes:**
    - **X-axis (Medial–Lateral):** From LTC toward the Medial Tibial Condyle
  (MTC).
    - **Y-axis (Proximal–Distal):** Points proximally along the tibia’s long
  axis toward the knee.
    - **Z-axis (Anterior–Posterior):** Points anteriorly; perpendicular to the
  plane defined by the X- and Y-axes.
  - **Interpretation:**
    - `+X` medial (toward the contralateral leg)
    - `+Y` proximal (toward the knee)
    - `+Z` anterior (forward)

## Analysis Approach
1. Load CSV data from static/dynamic trials
2. Extract 3D coordinates for each anatomical marker
3. Define local coordinate systems for thigh and shin segments
4. Estimate knee joint center:
   - Static: anatomical midpoint (MFEC↔LFEC)
   - Dynamic: sphere fitting of epicondyle motion
5. Visualize the biomechanical model in 3D

In [23]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

In [24]:
# ====== Load data from all CSV trials ======
import glob
import os

# Find all CSV files and sort them
csv_files = sorted(glob.glob("data/*_Bone Model.csv"))
print(f"Found {len(csv_files)} CSV files:\n")

def infer_trial_metadata(file_path, index):
    """Infer trial type/description from filename; fallback to generic types.
    Dataset-specific override: Trials 3 and 4 are dynamic even if filenames contain 'Static'.
    """
    fname = os.path.basename(file_path).lower()

    # Dataset-specific overrides per user clarification
    if index == 3 or "static trial 3" in fname:
        return "Dynamic - Flexion/Extension", "Dynamic knee bending (reclassified from dataset)"
    if index == 4 or "static trial 4" in fname:
        return "Dynamic - Varus/Valgus", "Medial-lateral knee motion (reclassified from dataset)"

    # Defaults by index (fallback)
    defaults = {
        1: ("Static - Flat Position", "Knee in extended/flat position"),
        2: ("Static - Flexed Position", "Knee in flexed position"),
        3: ("Dynamic - Flexion/Extension", "Dynamic knee bending"),
        4: ("Dynamic - Varus/Valgus", "Medial-lateral knee motion"),
    }
    # Heuristic from filename
    if "static" in fname:
        ttype = "Static"
        desc = "Static posture (no articulation); anatomical midpoint method"
    elif any(k in fname for k in ["flex", "extension"]):
        ttype = "Dynamic - Flexion/Extension"
        desc = "Dynamic knee bending"
    elif any(k in fname for k in ["varus", "valgus", "adduction", "abduction"]):
        ttype = "Dynamic - Varus/Valgus"
        desc = "Medial-lateral knee motion"
    else:
        ttype, desc = defaults.get(index, ("Unknown", "Unspecified trial"))
    return ttype, desc

# Build trial registry
trial_info = {}
for i, f in enumerate(csv_files, 1):
    trial_name = f"Trial {i}"
    ttype, desc = infer_trial_metadata(f, i)
    trial_info[trial_name] = {"file": f, "type": ttype, "description": desc}
    print(f"  {trial_name}: {f}")
    print(f"    Type: {ttype}")
    print(f"    Description: {desc}\n")

# Load all trials into a dictionary
all_trials_data = {}
for trial_name, info in trial_info.items():
    if info["file"]:
        df = pd.read_csv(info["file"], header=2)
        all_trials_data[trial_name] = {
            "dataframe": df,
            "type": info["type"],
            "description": info["description"],
            "file": info["file"]
        }
        print(f"✓ Loaded {trial_name}: {df.shape[0]} frames, {df.shape[1]} columns")

print(f"\n✓ All {len(all_trials_data)} trials loaded successfully")

Found 4 CSV files:

  Trial 1: data/Static Trial 1_Bone Model.csv
    Type: Static
    Description: Static posture (no articulation); anatomical midpoint method

  Trial 2: data/Static Trial 2_Bone Model.csv
    Type: Static
    Description: Static posture (no articulation); anatomical midpoint method

  Trial 3: data/Static Trial 3_Bone Model.csv
    Type: Dynamic - Flexion/Extension
    Description: Dynamic knee bending (reclassified from dataset)

  Trial 4: data/Static Trial 4_Bone Model.csv
    Type: Dynamic - Varus/Valgus
    Description: Medial-lateral knee motion (reclassified from dataset)

✓ Loaded Trial 1: 580 frames, 47 columns
✓ Loaded Trial 2: 582 frames, 47 columns
✓ Loaded Trial 3: 416 frames, 47 columns
✓ Loaded Trial 4: 588 frames, 47 columns

✓ All 4 trials loaded successfully


In [25]:
# ====== 2. Extract markers from all trials ======
def extract_triplet(df, marker_token):
    """Extract X, Y, Z triplet for a marker from the dataframe."""
    idx = None
    for i, c in enumerate(df.columns):
        if isinstance(c, str) and marker_token.lower() in c.lower():
            idx = i
            break
    if idx is None:
        return None
    triplet_cols = df.columns[idx:idx+3]
    sub = df.loc[:, triplet_cols].copy()
    for c in triplet_cols:
        sub[c] = pd.to_numeric(sub[c], errors="coerce")
    sub = sub.dropna(how="any")
    arr = sub.to_numpy(dtype=float)
    if arr.shape[1] != 3:
        return None
    return arr

# Define markers to extract
markers_to_extract = {
    # Shin/Tibia markers (use tibial landmarks)
    "MTC": "Subject 1:MTC",      # Medial Tibial Condyle
    "LTC": "Subject 1:LTC",      # Lateral Tibial Condyle  
    "uTD": "Subject 1:uTD",      # Upper Tibia (tuberosity)
    "MM": "Subject 1:MM",        # Medial Malleolus (ankle)
    "LM": "Subject 1:LM",        # Lateral Malleolus (ankle)
    
    # Thigh/Femur markers (use femoral landmarks)
    "MFEC": "Subject 1:MFEC",    # Medial Femoral Epicondyle
    "LFEC": "Subject 1:LFEC",    # Lateral Femoral Epicondyle
    "FH": "Subject 1:FH",        # Femoral Head
    "MC": "Subject 1:MC",        # Medial Condyle or Greater Trochanter
}

# Extract markers from each trial
trials_markers = {}

for trial_name, trial_data in all_trials_data.items():
    print(f"\n--- Processing {trial_name} ({trial_data['type']}) ---")
    df_raw = trial_data["dataframe"]
    
    data = {}
    for key, token in markers_to_extract.items():
        arr = extract_triplet(df_raw, token)
        if arr is None:
            print(f"  ❌ Could not parse {key}")
        else:
            data[key] = arr
            print(f"  ✓ {key}: {arr.shape[0]} frames")
    
    # Synchronize all markers to same number of frames
    if data:
        N = min(arr.shape[0] for arr in data.values())
        for k in data:
            data[k] = data[k][:N, :]
        
        trials_markers[trial_name] = {
            "data": data,
            "N": N,
            "type": trial_data["type"],
            "description": trial_data["description"]
        }
        print(f"  ✓ Synchronized to {N} frames")

print(f"\n✓ Marker extraction complete for all {len(trials_markers)} trials")


--- Processing Trial 1 (Static) ---
  ✓ MTC: 578 frames
  ✓ LTC: 578 frames
  ✓ uTD: 578 frames
  ✓ MM: 578 frames
  ✓ LM: 578 frames
  ✓ MFEC: 578 frames
  ✓ LFEC: 578 frames
  ✓ FH: 578 frames
  ✓ MC: 578 frames
  ✓ Synchronized to 578 frames

--- Processing Trial 2 (Static) ---
  ✓ MTC: 580 frames
  ✓ LTC: 580 frames
  ✓ uTD: 580 frames
  ✓ MM: 580 frames
  ✓ LM: 580 frames
  ✓ MFEC: 580 frames
  ✓ LFEC: 580 frames
  ✓ FH: 580 frames
  ✓ MC: 580 frames
  ✓ Synchronized to 580 frames

--- Processing Trial 3 (Dynamic - Flexion/Extension) ---
  ✓ MTC: 414 frames
  ✓ LTC: 414 frames
  ✓ uTD: 414 frames
  ✓ MM: 414 frames
  ✓ LM: 414 frames
  ✓ MFEC: 414 frames
  ✓ LFEC: 414 frames
  ✓ FH: 414 frames
  ✓ MC: 414 frames
  ✓ Synchronized to 414 frames

--- Processing Trial 4 (Dynamic - Varus/Valgus) ---
  ✓ MTC: 586 frames
  ✓ LTC: 586 frames
  ✓ uTD: 586 frames
  ✓ MM: 586 frames
  ✓ LM: 586 frames
  ✓ MFEC: 586 frames
  ✓ LFEC: 586 frames
  ✓ FH: 586 frames
  ✓ MC: 586 frames
  ✓ Synchr

In [26]:
# ====== 3. Compute frames and coordinates for all trials ======
def normalize_rows(v):
    """Normalize rows of array to unit vectors."""
    n = np.linalg.norm(v, axis=1, keepdims=True)
    n[n < 1e-12] = 1.0
    return v / n

def build_frame(p1, p2, p3):
    """
    Build orthogonal coordinate frame from 3 points.
    X-axis: from p1 to p2 (lateral->medial)
    Z-axis: perpendicular to plane defined by p1, p2, p3
    Y-axis: completes right-handed system
    Returns: R (rotation matrices), O (origins) — origin at p1.
    """
    x = normalize_rows(p2 - p1)
    z = normalize_rows(np.cross(x, p3 - p1))
    y = normalize_rows(np.cross(z, x))
    R = np.stack([x, y, z], axis=2)
    O = p1
    return R, O

def sphere_fit(points):
    """Fit sphere to cloud of 3D points using least squares."""
    P = points
    b = np.sum(P * P, axis=1).reshape(-1, 1)
    A = np.hstack([2 * P, np.ones((P.shape[0], 1))])
    x, *_ = np.linalg.lstsq(A, b, rcond=None)
    c = x[:3, 0]
    r = float(np.sqrt(np.mean(np.sum((P - c) ** 2, axis=1))))
    return c, r

# Process each trial
trials_results = {}

for trial_name, trial_markers in trials_markers.items():
    print(f"\n--- Computing {trial_name} ({trial_markers['type']}) ---")
    
    data = trial_markers["data"]
    N = trial_markers["N"]
    
    # Define SHIN coordinate system using tibial landmarks
    ankle_center = (data["MM"] + data["LM"]) / 2
    R_shin, O_shin = build_frame(data["LTC"], data["MTC"], ankle_center)
    
    # Define THIGH coordinate system using femoral landmarks  
    R_thigh, O_thigh = build_frame(data["LFEC"], data["MFEC"], data["FH"])
    
    # Decide method based on trial type
    ttype = trial_markers["type"].lower()
    is_static = "static" in ttype
    
    if is_static:
        # Anatomical midpoint of epicondyles (average across frames to reduce noise)
        center_series = 0.5 * (data["MFEC"] + data["LFEC"])
        center = np.mean(center_series, axis=0)
        r = float('nan')
        method = "anatomical-midpoint"
    else:
        # Sphere fit to epicondyle trajectories (functional method)
        thigh_cloud = np.vstack([data["MFEC"], data["LFEC"]])
        center, r = sphere_fit(thigh_cloud)
        method = "sphere-fit"
    
    # Use middle frame as reference for local coordinates
    frame = N // 2
    shinOrigin = O_shin[frame]
    thighOrigin = O_thigh[frame]
    Rsh = R_shin[frame]
    Rth = R_thigh[frame]
    
    # Calculate distances and local coordinates
    deltaShin = center - shinOrigin
    deltaThigh = center - thighOrigin
    distanceShinCOR = float(np.linalg.norm(deltaShin))
    distanceThighCOR = float(np.linalg.norm(deltaThigh))
    center_in_shin = Rsh.T @ (center - shinOrigin)
    center_in_thigh = Rth.T @ (center - thighOrigin)
    
    # Store results
    trials_results[trial_name] = {
        "data": data,
        "N": N,
        "type": trial_markers["type"],
        "description": trial_markers["description"],
        "method": method,
        "R_shin": R_shin,
        "O_shin": O_shin,
        "R_thigh": R_thigh,
        "O_thigh": O_thigh,
        "center": center,
        "radius": r,
        "frame": frame,
        "shinOrigin": shinOrigin,
        "thighOrigin": thighOrigin,
        "Rsh": Rsh,
        "Rth": Rth,
        "deltaShin": deltaShin,
        "deltaThigh": deltaThigh,
        "distanceShinCOR": distanceShinCOR,
        "distanceThighCOR": distanceThighCOR,
        "center_in_shin": center_in_shin,
        "center_in_thigh": center_in_thigh
    }
    
    if is_static:
        print("  ✓ Method: Anatomical midpoint (static)")
        print(f"  ✓ Joint center (avg MFEC↔LFEC): [{center[0]:.2f}, {center[1]:.2f}, {center[2]:.2f}] mm")
        print("  • Sphere fitting skipped for static trial")
    else:
        print("  ✓ Method: Sphere fit (dynamic)")
        print(f"  ✓ Joint center estimated at: [{center[0]:.2f}, {center[1]:.2f}, {center[2]:.2f}] mm")
        print(f"  ✓ Sphere radius: {r:.2f} mm")

print(f"\n✓ Analysis complete for all {len(trials_results)} trials")


--- Computing Trial 1 (Static) ---
  ✓ Method: Anatomical midpoint (static)
  ✓ Joint center (avg MFEC↔LFEC): [-23.41, 381.58, 30.43] mm
  • Sphere fitting skipped for static trial

--- Computing Trial 2 (Static) ---
  ✓ Method: Anatomical midpoint (static)
  ✓ Joint center (avg MFEC↔LFEC): [47.30, 353.53, 277.91] mm
  • Sphere fitting skipped for static trial

--- Computing Trial 3 (Dynamic - Flexion/Extension) ---
  ✓ Method: Sphere fit (dynamic)
  ✓ Joint center estimated at: [67.62, 327.85, 234.68] mm
  ✓ Sphere radius: 48.55 mm

--- Computing Trial 4 (Dynamic - Varus/Valgus) ---
  ✓ Method: Sphere fit (dynamic)
  ✓ Joint center estimated at: [57.51, 279.92, 129.87] mm
  ✓ Sphere radius: 47.37 mm

✓ Analysis complete for all 4 trials


In [27]:
# ====== 4. PRINT SUMMARY FOR ALL TRIALS ======
print("=" * 80)
print("KNEE JOINT CENTER ANALYSIS - SUMMARY OF ALL TRIALS")
print("=" * 80)

for trial_name, results in trials_results.items():
    print(f"\n{'=' * 80}")
    print(f"  {trial_name}: {results['type']}")
    print(f"  Method: {results['method']}")
    print(f"  {results['description']}")
    print(f"{'=' * 80}")
    print(f"Frames analyzed: {results['N']}")
    print(f"Reference frame: {results['frame']} (middle frame)")
    print()
    print(f"Knee Joint Center (COR) in lab coordinates:")
    print(f"  Position: [{results['center'][0]:.3f}, {results['center'][1]:.3f}, {results['center'][2]:.3f}] mm")
    rad = results['radius']
    if np.isfinite(rad) if isinstance(rad, float) else False:
        print(f"  Sphere radius: {rad:.3f} mm")
    else:
        print("  Sphere radius: N/A (static/anatomical method)")
    print()
    print(f"Distance from Thigh origin to COR: {results['distanceThighCOR']:.3f} mm")
    print(f"Distance from Shin origin to COR:  {results['distanceShinCOR']:.3f} mm")
    print()
    print(f"Δ (Thigh → COR) [X,Y,Z]: [{results['deltaThigh'][0]:.3f}, {results['deltaThigh'][1]:.3f}, {results['deltaThigh'][2]:.3f}] mm")
    print(f"Δ (Shin  → COR) [X,Y,Z]: [{results['deltaShin'][0]:.3f}, {results['deltaShin'][1]:.3f}, {results['deltaShin'][2]:.3f}] mm")
    print()
    print(f"COR in Thigh local frame: [{results['center_in_thigh'][0]:.3f}, {results['center_in_thigh'][1]:.3f}, {results['center_in_thigh'][2]:.3f}] mm")
    print(f"COR in Shin local frame:  [{results['center_in_shin'][0]:.3f}, {results['center_in_shin'][1]:.3f}, {results['center_in_shin'][2]:.3f}] mm")

print(f"\n{'=' * 80}")
print("COMPARISON SUMMARY")
print(f"{'=' * 80}")
print(f"{'Trial':<15} {'Type':<30} {'Method':<18} {'COR X':<10} {'COR Y':<10} {'COR Z':<10} {'Radius':<10}")
print("-" * 80)
for trial_name, results in trials_results.items():
    rad = results['radius']
    rad_str = f"{rad:>9.2f}" if (isinstance(rad, float) and np.isfinite(rad)) else f"{'N/A':>9}"
    print(f"{trial_name:<15} {results['type']:<30} {results['method']:<18} {results['center'][0]:>9.2f} {results['center'][1]:>9.2f} {results['center'][2]:>9.2f} {rad_str}")
print("=" * 80)

KNEE JOINT CENTER ANALYSIS - SUMMARY OF ALL TRIALS

  Trial 1: Static
  Method: anatomical-midpoint
  Static posture (no articulation); anatomical midpoint method
Frames analyzed: 578
Reference frame: 289 (middle frame)

Knee Joint Center (COR) in lab coordinates:
  Position: [-23.408, 381.583, 30.429] mm
  Sphere radius: N/A (static/anatomical method)

Distance from Thigh origin to COR: 44.329 mm
Distance from Shin origin to COR:  72.182 mm

Δ (Thigh → COR) [X,Y,Z]: [-42.192, -13.582, 0.711] mm
Δ (Shin  → COR) [X,Y,Z]: [-51.998, 49.787, 5.263] mm

COR in Thigh local frame: [44.329, -0.009, 0.011] mm
COR in Shin local frame:  [50.241, 51.810, 1.329] mm

  Trial 2: Static
  Method: anatomical-midpoint
  Static posture (no articulation); anatomical midpoint method
Frames analyzed: 580
Reference frame: 290 (middle frame)

Knee Joint Center (COR) in lab coordinates:
  Position: [47.299, 353.528, 277.914] mm
  Sphere radius: N/A (static/anatomical method)

Distance from Thigh origin to COR:

In [28]:
# ====== 5. 3D VISUALIZATION FOR EACH TRIAL ======
def trace_points(name, pts, color, size=3):
    """Create scatter trace for 3D points."""
    return go.Scatter3d(
        x=pts[:,0], y=pts[:,1], z=pts[:,2],
        mode='markers', marker=dict(size=size, color=color),
        name=name, opacity=0.8
)

def trace_arrow(name, origin, vec, color, width=6):
    """Create line trace for vector arrow."""
    p2 = origin + vec
    return go.Scatter3d(
        x=[origin[0], p2[0]],
        y=[origin[1], p2[1]],
        z=[origin[2], p2[2]],
        mode='lines', line=dict(width=width, color=color),
        name=name
)

def sphere_surface(center, radius, res=24):
    """Create semi-transparent sphere surface."""
    u = np.linspace(0, 2*np.pi, res)
    v = np.linspace(0, np.pi, res)
    x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
    y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
    z = center[2] + radius * np.outer(np.ones_like(u), np.cos(v))
    return go.Surface(x=x, y=y, z=z, opacity=0.25, showscale=False, name='COR sphere')

# Create visualization for each trial
for trial_name, results in trials_results.items():
    print(f"\nGenerating 3D visualization for {trial_name}...")
    
    data = results["data"]
    center = results["center"]
    r = results["radius"]
    shinOrigin = results["shinOrigin"]
    thighOrigin = results["thighOrigin"]
    Rsh = results["Rsh"]
    Rth = results["Rth"]
    deltaShin = results["deltaShin"]
    deltaThigh = results["deltaThigh"]
    
    traces = []
    
    # Plot anatomical markers
    # Thigh markers (red tones)
    traces += [trace_points("MFEC", data["MFEC"], "red", size=3)]
    traces += [trace_points("LFEC", data["LFEC"], "darkred", size=3)]
    traces += [trace_points("FH", data["FH"], "pink", size=2)]
    
    # Shin markers (blue tones)
    traces += [trace_points("MTC", data["MTC"], "blue", size=3)]
    traces += [trace_points("LTC", data["LTC"], "darkblue", size=3)]
    traces += [trace_points("MM", data["MM"], "lightblue", size=2)]
    traces += [trace_points("LM", data["LM"], "cyan", size=2)]
    
    # Coordinate system visualization (at reference frame)
    scale = 40.0
    traces += [
        trace_arrow("Shin X",  shinOrigin,  scale * Rsh[:,0], "red"),
        trace_arrow("Shin Y",  shinOrigin,  scale * Rsh[:,1], "green"),
        trace_arrow("Shin Z",  shinOrigin,  scale * Rsh[:,2], "blue"),
        trace_arrow("Thigh X", thighOrigin, scale * Rth[:,0], "orange"),
        trace_arrow("Thigh Y", thighOrigin, scale * Rth[:,1], "brown"),
        trace_arrow("Thigh Z", thighOrigin, scale * Rth[:,2], "purple"),
]
    
    # Center of rotation and optional sphere
    traces += [go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]],
                            mode='markers+text',
                            marker=dict(size=8, color='black', symbol='diamond'),
                            text=["Knee COR"],
                            textposition="top center",
                            name="COR")]
    if isinstance(r, float) and np.isfinite(r) and r > 1e-6:
        traces += [sphere_surface(center, r)]
    
    # Connection lines showing distances
    traces += [
        trace_arrow("Shin→COR",  shinOrigin,  center - shinOrigin,  "black", width=4),
        trace_arrow("Thigh→COR", thighOrigin, center - thighOrigin, "brown", width=4),
]
    
    # Per-axis component vectors (from shin origin)
    traces += [
        trace_arrow("ΔX shin", shinOrigin, np.array([deltaShin[0], 0, 0]), "red", width=2),
        trace_arrow("ΔY shin", shinOrigin, np.array([0, deltaShin[1], 0]), "green", width=2),
        trace_arrow("ΔZ shin", shinOrigin, np.array([0, 0, deltaShin[2]]), "blue", width=2),
]
    
    layout = go.Layout(
        scene=dict(
            xaxis_title="X (mm)", 
            yaxis_title="Y (mm)", 
            zaxis_title="Z (mm)",
            aspectmode='data'
        ),
        title=f"{trial_name}: {results['type']}<br><sub>{results['description']}</sub>",
        showlegend=True, 
        height=800,
        width=1200
)
    
    fig = go.Figure(data=traces, layout=layout)
    fig.show()
    
print("\n✓ All visualizations generated")


Generating 3D visualization for Trial 1...



Generating 3D visualization for Trial 2...



Generating 3D visualization for Trial 3...



Generating 3D visualization for Trial 4...



✓ All visualizations generated
